In [39]:
import pandas as pd
import numpy as np

import tensorflow as tf
import tensorflow_hub as hub

from sklearn.neighbors import NearestNeighbors


import warnings
warnings.filterwarnings("ignore")
pd.set_option('display.max_colwidth', None)

In [18]:
symptomsDF = pd.read_csv('C:/Users/HP/Documents/AIML/DiagnoSmart/Dataset/dataset.csv')
symptomsDF.head(2)

,Disease,Symptom_1,Symptom_2,Symptom_3,Symptom_4,Symptom_5,Symptom_6,Symptom_7,Symptom_8,Symptom_9,Symptom_10,Symptom_11,Symptom_12,Symptom_13,Symptom_14,Symptom_15,Symptom_16,Symptom_17
0,Fungal infection,itching,skin_rash,nodal_skin_eruptions,dischromic _patches,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Fungal infection,skin_rash,nodal_skin_eruptions,dischromic _patches,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [19]:
symptomsDF = symptomsDF.replace(np.NaN, 'None')

cols = list(symptomsDF.columns)
for i in cols[1:]:
  symptomsDF[i] = symptomsDF[i].str.split("_")
symptomsDF.head(2)

,Disease,Symptom_1,Symptom_2,Symptom_3,Symptom_4,Symptom_5,Symptom_6,Symptom_7,Symptom_8,Symptom_9,Symptom_10,Symptom_11,Symptom_12,Symptom_13,Symptom_14,Symptom_15,Symptom_16,Symptom_17
0,Fungal infection,[itching],"[ skin, rash]","[ nodal, skin, eruptions]","[ dischromic , patches]",[None],[None],[None],[None],[None],[None],[None],[None],[None],[None],[None],[None],[None]
1,Fungal infection,"[ skin, rash]","[ nodal, skin, eruptions]","[ dischromic , patches]",[None],[None],[None],[None],[None],[None],[None],[None],[None],[None],[None],[None],[None],[None]


In [20]:
symptomsDF['combined_symptoms'] = symptomsDF['Symptom_1'] + symptomsDF['Symptom_2'] + symptomsDF['Symptom_3'] + symptomsDF['Symptom_4'] + symptomsDF['Symptom_5']
symptomsDF.drop(columns=cols[1:],inplace=True)
symptomsDF.head(2)

,Disease,combined_symptoms
0,Fungal infection,"[itching, skin, rash, nodal, skin, eruptions, dischromic , patches, None]"
1,Fungal infection,"[ skin, rash, nodal, skin, eruptions, dischromic , patches, None, None]"


In [22]:
testDF = symptomsDF.copy()
testDF['combined_symptoms'] = testDF['combined_symptoms'].apply(lambda x: ' '.join(x))
testDF.head(2)

,Disease,combined_symptoms
0,Fungal infection,itching skin rash nodal skin eruptions dischromic patches None
1,Fungal infection,skin rash nodal skin eruptions dischromic patches None None


In [23]:
def clean_duplicate(text):
  words = text.split()
  unique_words = set()
  result = []
  for word in words:
      if word not in unique_words:
          unique_words.add(word)
          result.append(word)

  return ' '.join(result)

testDF['combined_symptoms'] = testDF['combined_symptoms'].apply(clean_duplicate)
testDF.head(2)

,Disease,combined_symptoms
0,Fungal infection,itching skin rash nodal eruptions dischromic patches None
1,Fungal infection,skin rash nodal eruptions dischromic patches None


In [24]:
testDF = testDF.drop_duplicates()
testDF.shape

(235, 2)

In [25]:
testDF['Disease'].unique()

array(['Fungal infection', 'Allergy', 'GERD', 'Chronic cholestasis',
       'Drug Reaction', 'Peptic ulcer diseae', 'AIDS', 'Diabetes ',
       'Gastroenteritis', 'Bronchial Asthma', 'Hypertension ', 'Migraine',
       'Cervical spondylosis', 'Paralysis (brain hemorrhage)', 'Jaundice',
       'Malaria', 'Chicken pox', 'Dengue', 'Typhoid', 'hepatitis A',
       'Hepatitis B', 'Hepatitis C', 'Hepatitis D', 'Hepatitis E',
       'Alcoholic hepatitis', 'Tuberculosis', 'Common Cold', 'Pneumonia',
       'Dimorphic hemmorhoids(piles)', 'Heart attack', 'Varicose veins',
       'Hypothyroidism', 'Hyperthyroidism', 'Hypoglycemia',
       'Osteoarthristis', 'Arthritis',
       '(vertigo) Paroymsal  Positional Vertigo', 'Acne',
       'Urinary tract infection', 'Psoriasis', 'Impetigo'], dtype=object)

In [26]:
symptomsDF.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4920 entries, 0 to 4919
Data columns (total 2 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   Disease            4920 non-null   object
 1   combined_symptoms  4920 non-null   object
dtypes: object(2)
memory usage: 77.0+ KB


In [28]:
diseaseDescriptionDF = pd.read_csv('C:/Users/HP/Documents/AIML/DiagnoSmart/Dataset/symptom_Description.csv')

In [146]:
diseaseDescriptionDF.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 41 entries, 0 to 40
Data columns (total 2 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   Disease      41 non-null     object
 1   Description  41 non-null     object
dtypes: object(2)
memory usage: 784.0+ bytes


In [32]:
diseasePrecautionDF = pd.read_csv('C:/Users/HP/Documents/AIML/DiagnoSmart/Dataset/symptom_precaution.csv')

In [33]:
diseasePrecautionDF.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 41 entries, 0 to 40
Data columns (total 5 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   Disease       41 non-null     object
 1   Precaution_1  41 non-null     object
 2   Precaution_2  41 non-null     object
 3   Precaution_3  40 non-null     object
 4   Precaution_4  40 non-null     object
dtypes: object(5)
memory usage: 1.7+ KB


In [34]:
merged_df = pd.merge(diseaseDescriptionDF, diseasePrecautionDF, on='Disease', how = 'left')

In [35]:
merged_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 41 entries, 0 to 40
Data columns (total 6 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   Disease       41 non-null     object
 1   Description   41 non-null     object
 2   Precaution_1  38 non-null     object
 3   Precaution_2  38 non-null     object
 4   Precaution_3  37 non-null     object
 5   Precaution_4  37 non-null     object
dtypes: object(6)
memory usage: 2.1+ KB


In [36]:
merged_df.set_index('Disease',inplace=True)
merged_df.head(2)

,Description,Precaution_1,Precaution_2,Precaution_3,Precaution_4
Disease,,,,,
Drug Reaction,An adverse drug reaction (ADR) is an injury caused by taking medication. ADRs may occur following a single dose or prolonged administration of a drug or result from the combination of two or more drugs.,stop irritation,consult nearest hospital,stop taking drug,follow up
Malaria,An infectious disease caused by protozoan parasites from the Plasmodium family that can be transmitted by the bite of the Anopheles mosquito or by a contaminated needle or transfusion. Falciparum malaria is the most deadly type.,Consult nearest hospital,avoid oily food,avoid non veg food,keep mosquitos out


In [37]:
merged_df = merged_df.replace(np.NaN, 'None')

In [38]:
module_url = "https://tfhub.dev/google/universal-sentence-encoder/4"
model = hub.load(module_url)
embed = model(testDF['combined_symptoms'])

In [40]:
neigh = NearestNeighbors(n_neighbors=1)

neigh.fit(embed)

NearestNeighbors(n_neighbors=1)

In [44]:
symptoms = input('Enter your symptoms: ')
symptoms

'my skin is itching and i have rashes on my vody'

In [45]:
emd = model([symptoms])
nei = neigh.kneighbors(emd,return_distance=False)[0][0]
disease_name = testDF['Disease'].iloc[nei]
print('Predicted Diagnosis: ', disease_name)

Predicted Diagnosis:  Fungal infection


In [48]:
description = merged_df['Description'].loc[disease_name]
print(disease_name,':', description)

Fungal infection : In humans, fungal infections occur when an invading fungus takes over an area of the body and is too much for the immune system to handle. Fungi can live in the air, soil, water, and plants. There are also some fungi that live naturally in the human body. Like many microbes, there are helpful fungi and harmful fungi.


In [54]:
Precauions = list(merged_df[['Precaution_1','Precaution_2','Precaution_3','Precaution_4']].loc[disease_name])
print('Precauions')
for count, precaution in enumerate(Precauions, start=1):
  print(count,'.',precaution)

Precauions
1 . bath twice
2 . use detol or neem in bathing water
3 . keep infected area dry
4 . use clean cloths
